# cloud-object-store-housekeeping

Housekeeping (delete, list, ...) on any S3 compliant Cloud Object Storage

In [ ]:
!pip install aiobotocore==2.0.1
!pip install botocore==1.22.8
!pip install s3fs==2021.11.1

In [ ]:
import logging
import os
import sys
import re
import s3fs

In [ ]:
access_key_id = os.environ.get('access_key_id')
secret_access_key = os.environ.get('secret_access_key')
endpoint = os.environ.get('endpoint')
bucket_name = os.environ.get('bucket_name')

# default: ls, operation in rm (delete), ls (list), walk (walk the tree)
operation = os.environ.get('operation', 'ls')

# default: sysout, in "ls", "walk" operation, return list on sysout or as file (provide file name instead of sysout)
return_mode = os.environ.get('return_mode', 'sysout')

# defaul: empty string, file/folder to operate on
path = os.environ.get('path', "")

In [ ]:
parameters = list(
    map(lambda s: re.sub('$', '"', s),
        map(
            lambda s: s.replace('=', '="'),
            filter(
                lambda s: s.find('=') > -1 and bool(re.match(r'[A-Za-z0-9_]*=[.\/A-Za-z0-9]*', s)),
                sys.argv
            )
    )))

for parameter in parameters:
    logging.warning('Parameter: ' + parameter)
    exec(parameter)

In [ ]:
s3 = s3fs.S3FileSystem(
    anon=False,
    key=access_key_id,
    secret=secret_access_key,
    client_kwargs={'endpoint_url': endpoint}
)

In [ ]:
if operation == 'ls':
    result_list = s3.ls(bucket_name)
    if return_mode == 'sysout':
        print(result_list)
elif operation == 'walk':
    result_list = s3.walk(bucket_name + '/' + path,  maxdepth=100)
    if return_mode == 'sysout':
        for item in result_list:
            print(item)
elif operation == 'rm':
    s3.rm(bucket_name + '/' + path, recursive=True, maxdepth=100)
else:
    raise 'operation unknown: {}'.format(operation)